<a href="https://colab.research.google.com/github/0xVolt/whats-up-doc/blob/main/test/notebooks/model-blending/blend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Merging CodeLLMs to Create an Efficant Low-Memory Quantized Model for `whats-up-doc` using the TIES Method

## Download and Install `mergekit`

In [1]:
import os

dirName = "mergekit"
cwd = os.getcwd()

concatDirPath = os.path.join(cwd, dirName)

if not os.path.exists(concatDirPath):
    !git clone https://github.com/cg123/mergekit.git
    !cd mergekit && pip install -q -e .

Cloning into 'mergekit'...
remote: Enumerating objects: 2265, done.
remote: Counting objects: 100% (1340/1340), done.
remote: Compressing objects: 100% (514/514), done.
remote: Total 2265 (delta 1068), reused 939 (delta 825), pack-reused 925
Receiving objects: 100% (2265/2265), 638.44 KiB | 16.80 MiB/s, done.
Resolving deltas: 100% (1584/1584), done.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
keras-cv 0.8.2 requires keras-core, which is not installed.
keras-nlp 0.9.3 requires keras-core, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.6 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, 

## Create the YAML Config File to Merge Models with SLERP

In [2]:
import os
import yaml
from transformers import AutoModelWithLMHead, AutoTokenizer, pipeline

2024-05-19 11:19:52.194007: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-19 11:19:52.194105: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-19 11:19:52.335750: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
%pip install huggingface-cli

Note: you may need to restart the kernel to use updated packages.


In [4]:
from kaggle_secrets import UserSecretsClient
userSecrets = UserSecretsClient()
HF_WRITE_TOKEN = userSecrets.get_secret("HF_WRITE_TOKEN")

### Write Config Script

In [12]:
# Set the resultant model's name
MODEL_NAME = 'whats-up-llamas-ties'

MODEL_1 = "meta-llama/Meta-Llama-3-8B-Instruct"
MODEL_2 = "codellama/CodeLlama-7b-Instruct-hf"

OUTPUT_DIR = "whats-up-llamas-ties"

#### TIES YAML Config Creation

In [13]:
yamlConfigTIESLlamas = f"""
models:
  - model: {MODEL_1}  # no parameters necessary for base model
  - model: {MODEL_2}
    parameters:
      density: 0.5
      weight: 0.5
merge_method: ties
base_model: {MODEL_1}
parameters:
  normalize: true
  int8_mask: true
dtype: float16

"""

### Save Config Script

In [14]:
# Save the YAML configuration to a file
yamlFileName = "config.yaml"
with open(yamlFileName, "w") as f:
    f.write(yamlConfigTIESLlamas)

## Merge Models

In [15]:
cmd = f"mergekit-yaml {yamlFileName} {OUTPUT_DIR} --allow-crimes --copy-tokenizer --out-shard-size 1B --low-cpu-memory --write-model-card --lazy-unpickle"
!{cmd}

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_errors.py", line 304, in hf_raise_for_status
    response.raise_for_status()
  File "/opt/conda/lib/python3.10/site-packages/requests/models.py", line 1021, in raise_for_status
    raise HTTPError(http_error_msg, response=self)
requests.exceptions.HTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct/resolve/main/config.json

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/transformers/utils/hub.py", line 398, in cached_file
    resolved_file = hf_hub_download(
  File "/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_validators.py", line 119, in _inner_fn
    return fn(*args, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/huggingface_hub/file_download.py", line 1403, in hf_hub_download
    raise hea